In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
#df = pd.read_csv('/kaggle/input/tencentforuse/edit')
new_df=pd.read_csv('/kaggle/input/newclustering/newclasstering.csv')
new_df

In [2]:
#normalized_df=(df_new-df_new.mean())/df_new.std()
#new_df=new_df.fillna(new_df.mean())
new_df[['status']]=new_df[['status']].fillna(round(new_df.mean()))
new_df=new_df.fillna(0)
normalized=pd.DataFrame()
normalized['status']=(new_df['status']-new_df['status'].mean())/new_df['status'].std()
normalized['play number']=(new_df['play number']-new_df['play number'].mean())/new_df['play number'].std()
normalized['score']=(new_df['score']-new_df['score'].mean())/new_df['score'].std()
#normalized['year']=new_df['year']-2015
normalized['year']=(new_df['year']-new_df['year'].mean())/new_df['year'].std()
#normalized['topic']=new_df['topic']-7
normalized['topic']=(new_df['topic']-new_df['topic'].mean())/new_df['topic'].std()
normalized['1']=(new_df['1']-new_df['1'].mean())/new_df['1'].std()
normalized['2']=(new_df['2']-new_df['2'].mean())/new_df['2'].std()
normalized['sum']=(new_df['sum']-new_df['sum'].mean())/new_df['sum'].std()
normalized['play cat']=new_df['play cat']
df_last1=normalized
df_last1
#new_df

In [3]:
#from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.model_selection import train_test_split
#test=df_last1[['status','score','year', 'topic','1','2','sum']]

In [4]:
##https://blog.csdn.net/weixin_42029733/article/details/89922575
from itertools import combinations
def combine(temp_list,n):
    temp_list2 = []
    for c in combinations(temp_list, n):
        temp_list2.append(list(c))
    return temp_list2
test=['status','year', 'topic','1','2']
end_list=[]
for i in range(len(test)):
    end_list.extend(list(combine(test,i)))
del end_list[0]
end_list.append(test)
#end_list

In [5]:
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [6]:
#https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html#sphx-glr-auto-examples-model-selection-plot-grid-search-digits-py
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [7]:
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
dataX=df_last1[test]
dataY=df_last1["play cat"]
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio)
# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

In [8]:
#gamma=[0.03,0.1,0.3,1]
#C=[0.1,1,10,100]
C=[0.1*i for i in range(10,51)]
gamma=[i for i in range(4,30)]
record=[]
for g in gamma:
#for c in C:
    rbf = svm.SVC(kernel='rbf', gamma=g, C=2.3).fit(X_train, y_train)
    rbf_pred = rbf.predict(X_val)
    accuracy = accuracy_score(y_val, rbf_pred)
    record.append(accuracy)
import matplotlib.pyplot as plt
plt.plot(gamma, record)
plt.title('validation accuracy vs SVM(rbf kernel) gamma')
plt.ylabel('Accuracy')
plt.xlabel('Gamma')
plt.legend()
plt.show()

In [9]:
result=[]
gamma=[0.5,1.0,1.5,2.0]
C=[1.0,1.2,1.4,1.6,1.8,2.0]
for s in end_list:
    #X=X_train[s]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
    rbf = svm.SVC(kernel='rbf', gamma=12, C=2.3).fit(X_train[s], y_train)
    rbf_pred = rbf.predict(X_val[s])
    accuracy = accuracy_score(y_val, rbf_pred)
    result.append((s,accuracy))
result.sort(key= lambda x:x[1],reverse=True)
for outcome in result:
    print(outcome)


In [10]:
s=['status', 'year', '1', '2']
rbf = svm.SVC(kernel='rbf', gamma=12, C=2.3).fit(X_train[s], y_train)
rbf.fit(X_train[s], y_train)
rbf_pred = rbf.predict(X_test[s])
from sklearn.metrics import confusion_matrix
x=confusion_matrix(y_test, rbf_pred)
accuracy = accuracy_score(y_test, rbf_pred)
print(x)
accuracy

In [11]:
##https://towardsdatascience.com/multiclass-classification-with-support-vector-machines-svm-kernel-trick-kernel-functions-f9d5377d6f02
rbf = svm.SVC(kernel='rbf', gamma=2, C=1.8).fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=10).fit(X_train, y_train)
sig = svm.SVC(kernel='sigmoid', C=10).fit(X_train, y_train)
linear = svm.SVC(kernel='linear', C=10).fit(X_train, y_train)

In [12]:
poly_pred = poly.predict(X_test)
rbf_pred = rbf.predict(X_test)
sig_pred=sig.predict(X_test)
linear_pred=linear.predict(X_test)

In [13]:
poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

sig_accuracy = accuracy_score(y_test, sig_pred)
sig_f1 = f1_score(y_test, sig_pred, average='weighted')
print('Accuracy (Sig Kernel): ', "%.2f" % (sig_accuracy*100))
print('F1 (Sig Kernel): ', "%.2f" % (sig_f1*100))

linear_accuracy = accuracy_score(y_test, linear_pred)
linear_f1 = f1_score(y_test, linear_pred, average='weighted')
print('Accuracy (Linear Kernel): ', "%.2f" % (linear_accuracy*100))
print('F1 (Linear Kernel): ', "%.2f" % (linear_f1*100))


In [14]:
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt 
accu=[]
n_neighbors=range(10,60)
matric=['euclidean','manhattan','minkowski','hamming']
#for n in n_neighbors:
for m in ['uniform','distance']:
    neigh = KNeighborsClassifier(weights=m,n_neighbors=24)
    neigh.fit(X_train, y_train)
    neigh_pred=neigh.predict(X_val)
    neigh_accuracy = accuracy_score(y_val, neigh_pred)
    print(m,neigh_accuracy)
#import matplotlib.pyplot as plt
#plt.plot(n_neighbors, accu)
#plt.title('validation accuracy vs Number of neighbors(K) in KNN')
#plt.ylabel('Accuracy')
#plt.xlabel('Number of neighbors')
#plt.legend()
#plt.show()

In [15]:
result=[]
for s in end_list:
    #X=df_last1[s]
    #y=df_last1["play cat"]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    rbf = KNeighborsClassifier(weights='distance',n_neighbors=24)
    rbf.fit(X_train[s], y_train)
    rbf_pred = rbf.predict(X_val[s])
    accuracy = accuracy_score(y_val, rbf_pred)
    result.append((s,accuracy))
result.sort(key= lambda x:x[1],reverse=True)
for outcome in result:
    print(outcome)

In [16]:
s=['status', 'year', '1']
rbf = KNeighborsClassifier(weights='distance',n_neighbors=24)
rbf.fit(X_train[s], y_train)
rbf_pred = rbf.predict(X_test[s])
from sklearn.metrics import confusion_matrix
x=confusion_matrix(y_test, rbf_pred)
accuracy = accuracy_score(y_test, rbf_pred)
print(x)
accuracy

In [17]:
from sklearn.linear_model import RidgeClassifier
#reg =RidgeClassifier()
#reg = linear_model.Lasso(alpha=0.9)
#reg.fit(X_train, y_train)
#reg_pred=reg.predict(X_test)
#print(reg_pred)
#reg_accuracy = accuracy_score(y_test, reg_pred)
#reg_accuracy

In [18]:
result=[]
alpha=[1,10,100,1000,10000,100000,1000000]
for s in end_list:
    #X=df_last1[s]
    #y=df_last1["play cat"]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    rbf = RidgeClassifier(190)
    rbf.fit(X_train[s], y_train)
    rbf_pred = rbf.predict(X_test[s])
    accuracy = accuracy_score(y_test, rbf_pred)
    result.append((s,accuracy))
result.sort(key= lambda x:x[-1],reverse=True)
for outcome in result:
    print(outcome)

In [19]:
s=['status', '1']
rbf = RidgeClassifier(190)
rbf.fit(X_train[s], y_train)
rbf_pred = rbf.predict(X_test[s])
from sklearn.metrics import confusion_matrix
x=confusion_matrix(y_test, rbf_pred)
accuracy = accuracy_score(y_test, rbf_pred)
print(x)
accuracy


In [20]:
from sklearn.linear_model import RidgeClassifier
result=[]
alpha=[i for i in range(1,1000)]
for a in alpha:
    #X=df_last1[s]
    #y=df_last1["play cat"]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    rbf = RidgeClassifier(alpha=a)
    rbf.fit(X_train, y_train)
    rbf_pred = rbf.predict(X_val)
    accuracy = accuracy_score(y_val, rbf_pred)
    result.append(accuracy)
import matplotlib.pyplot as plt
plt.plot(alpha, result)
plt.title('validation accuracy vs SVM(rbf kernel) gamma')
plt.ylabel('Accuracy')
plt.xlabel('Gamma')
plt.legend()
plt.show()
#import matplotlib.pyplot as plt
#plt.plot(range(-5,5), result)
#plt.legend()
#plt.show()

In [21]:
from sklearn.linear_model import LogisticRegression
result=[]
C=[0.001*i for i in range(1,15)]
for c in C:
    #X=df_last1[s]
    #y=df_last1["play cat"]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    rbf = LogisticRegression(C=c,random_state=0)
    rbf.fit(X_train, y_train)
    rbf_pred = rbf.predict(X_test)
    accuracy = accuracy_score(y_test, rbf_pred)
    result.append(accuracy)
import matplotlib.pyplot as plt
plt.plot(C, result)
plt.title('validation accuracy vs SVM(rbf kernel) gamma')
plt.ylabel('Accuracy')
plt.xlabel('Gamma')
plt.legend()
plt.show()

In [23]:
from sklearn.linear_model import LogisticRegression
result=[]
for s in end_list:
    #X=df_last1[s]
    #y=df_last1["play cat"]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    rbf = LogisticRegression(C=0.006
                             ,random_state=0)
    rbf.fit(X_train[s], y_train)
    rbf_pred = rbf.predict(X_test[s])
    accuracy = accuracy_score(y_test, rbf_pred)
    result.append((s,accuracy))
result.sort(key= lambda x:x[-1],reverse=True)
for outcome in result:
    print(outcome)

In [24]:
s=['status', '1']
rbf = LogisticRegression(C=0.005,random_state=0)
rbf.fit(X_train[s], y_train)
rbf_pred = rbf.predict(X_test[s])
from sklearn.metrics import confusion_matrix
x=confusion_matrix(y_test, rbf_pred)
accuracy = accuracy_score(y_test, rbf_pred)
print(x)
accuracy

In [25]:
from sklearn import tree
result=[]
for s in end_list:
    #X=df_last1[s]
    #y=df_last1["play cat"]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X_train[s], y_train)
    predictions = clf.predict(X_val[s])
    accuracy = accuracy_score(y_val, predictions)
    result.append((s,accuracy))
result.sort(key= lambda x:x[1],reverse=True)
for outcome in result:
    print(outcome)
#tree.plot_tree(clf)

In [26]:
s=['status', 'year', '1']
rbf = tree.DecisionTreeClassifier()
rbf.fit(X_train[s], y_train)
rbf_pred = rbf.predict(X_test[s])
from sklearn.metrics import confusion_matrix
x=confusion_matrix(y_test, rbf_pred)
accuracy = accuracy_score(y_test, rbf_pred)
print(x)
accuracy

In [27]:
from sklearn.ensemble import RandomForestClassifier
result=[]
max_depth=range(1,101)
n_estimators=range(40,141)
max_features=range(1,6)
#for depth in max_depth:
for n in n_estimators:
#for features in max_features:
        clf = RandomForestClassifier(criterion='entropy',max_depth=22,n_estimators=n,max_features=2)
        clf = clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        result.append(accuracy)
import matplotlib.pyplot as plt
plt.plot(n_estimators, result)
plt.title('validation accuracy vs maximum number RandomForest terminated')
plt.ylabel('Accuracy')
plt.xlabel('Maximum number')
plt.legend()
plt.show()

In [28]:
from sklearn.ensemble import RandomForestClassifier
result=[]
max_depth=range(20,30)
n_estimators=range(20,30)
max_features=range(1,6)
for s in end_list:
    #X=df_last1[s]
    #y=df_last1["play cat"]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    clf = RandomForestClassifier(criterion='entropy',max_depth=22,n_estimators=40,max_features=min(2,len(s)))
    clf = clf.fit(X_train[s], y_train)
    predictions = clf.predict(X_val[s])
    accuracy = accuracy_score(y_val, predictions)
    result.append((s,accuracy))
result.sort(key= lambda x:x[-1],reverse=True)
for outcome in result:
    print(outcome)
    

In [29]:
s=['status', 'year', '1']
rbf = RandomForestClassifier(criterion='entropy',max_depth=22,n_estimators=40,max_features=min(2,len(s)))
rbf.fit(X_train[s], y_train)
rbf_pred = rbf.predict(X_test[s])
from sklearn.metrics import confusion_matrix
x=confusion_matrix(y_test, rbf_pred)
accuracy = accuracy_score(y_test, rbf_pred)
print(x)
accuracy

In [30]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
result=[]
max_feature=range(1,6)
n_estimators=range(1,51)
for f in max_feature:
    for n in n_estimators:
        #X=df_last1[s]
        #y=df_last1["play cat"]
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
        clf = BaggingClassifier(KNeighborsClassifier(),max_samples=1.0, max_features=f,n_estimators=n)
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        result.append((f,n,accuracy))
result.sort(key= lambda x:x[-1],reverse=True)
for outcome in result:
    print(outcome)

In [31]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
result=[]
for s in end_list:
        #X=df_last1[s]
        #y=df_last1["play cat"]
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    clf = BaggingClassifier(KNeighborsClassifier(),max_samples=1.0, max_features=min(2,len(s)),n_estimators=27)
    clf.fit(X_train[s], y_train)
    predictions = clf.predict(X_val[s])
    accuracy = accuracy_score(y_val, predictions)
    result.append((s,accuracy))
result.sort(key= lambda x:x[-1],reverse=True)
for outcome in result:
    print(outcome)

In [32]:
s=['status', 'year', 'topic', '2']
rbf =BaggingClassifier(KNeighborsClassifier(),max_samples=1.0, max_features=min(2,len(s)),n_estimators=27)
rbf.fit(X_train[s], y_train)
rbf_pred = rbf.predict(X_test[s])
from sklearn.metrics import confusion_matrix
x=confusion_matrix(y_test, rbf_pred)
accuracy = accuracy_score(y_test, rbf_pred)
print(x)
accuracy

In [33]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier
n_estimators =range(110,211)
result=[]
for n in n_estimators:
    #for n in n_estimators:
        #X=df_last1[s]
        #y=df_last1["play cat"]
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    clf = AdaBoostClassifier(n_estimators=n)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    result.append(accuracy)
import matplotlib.pyplot as plt
plt.plot(range(110,211), result)
plt.title('validation accuracy vs maximum number Adaboosting terminated')
plt.ylabel('Accuracy')
plt.xlabel('Maximum number')
plt.legend()
plt.show()

In [34]:
from sklearn.model_selection import cross_val_score
#from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier
result=[]
for s in end_list:
    #X=df_last1[s]
    #y=df_last1["play cat"]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    clf = AdaBoostClassifier(n_estimators=155)
    clf.fit(X_train[s], y_train)
    predictions = clf.predict(X_val[s])
    accuracy = accuracy_score(y_val, predictions)
    result.append((s,accuracy))
result.sort(key= lambda x:x[1],reverse=True)
for outcome in result:
    print(outcome)

In [35]:
s=['status', 'year', 'topic', '2']
rbf = AdaBoostClassifier(n_estimators=155)
rbf.fit(X_train[s], y_train)
rbf_pred = rbf.predict(X_test[s])
from sklearn.metrics import confusion_matrix
x=confusion_matrix(y_test, rbf_pred)
accuracy = accuracy_score(y_test, rbf_pred)
print(x)
accuracy

In [36]:
from sklearn.ensemble import GradientBoostingClassifier
result=[]
for s in end_list:
    #X=df_last1[s]
    #y=df_last1["play cat"]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    clf = GradientBoostingClassifier(n_estimators=22, learning_rate=1.0,max_depth=3, random_state=0)
    clf.fit(X_train[s], y_train)
    predictions = clf.predict(X_val[s])
    accuracy = accuracy_score(y_val, predictions)
    result.append((s,accuracy))
result.sort(key= lambda x:x[1],reverse=True)
for outcome in result:
    print(outcome)

In [37]:
from sklearn.ensemble import GradientBoostingClassifier
max_depth=[1,2,3,4,5,6,7]
n_estimators=range(1,101)
result=[]
for n in n_estimators:
    #X=df_last1[s]
    #y=df_last1["play cat"]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    clf = GradientBoostingClassifier(n_estimators=n, learning_rate=1.0,max_depth=3, random_state=0)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    result.append(accuracy)
#result.sort(key= lambda x:x[1],reverse=True)
#for outcome in result:
    #print(outcome)
import matplotlib.pyplot as plt
plt.plot(range(1,101), result)
plt.title('validation accuracy vs maximum number Gradientboosting terminated')
plt.ylabel('Accuracy')
plt.xlabel('Maximum number')
plt.legend()
plt.show()

In [38]:
from sklearn.ensemble import GradientBoostingClassifier
max_depth=range(1,11)
n_estimators=range(1,101)
result=[]
for d in max_depth:
    #X=df_last1[s]
    #y=df_last1["play cat"]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    clf = GradientBoostingClassifier(n_estimators=22, learning_rate=1.0,max_depth=d, random_state=0)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    result.append(accuracy)
#result.sort(key= lambda x:x[1],reverse=True)
#for outcome in result:
    #print(outcome)
import matplotlib.pyplot as plt
plt.plot(max_depth, result)
plt.title('validation accuracy vs maximum depth Gradientboosting terminated')
plt.ylabel('Accuracy')
plt.xlabel('Maximum depth')
plt.legend()
plt.show()

In [39]:
s=['status', 'year', 'topic', '2']
rbf = GradientBoostingClassifier(n_estimators=22, learning_rate=1.0,max_depth=3, random_state=0)
rbf.fit(X_train[s], y_train)
rbf_pred = rbf.predict(X_test[s])
from sklearn.metrics import confusion_matrix
x=confusion_matrix(y_test, rbf_pred)
accuracy = accuracy_score(y_test, rbf_pred)
print(x)
accuracy